# EDA

# Import libraries and load datasets

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.0 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import re
import geopandas as gpd
from shapely.geometry import Polygon, Point
from folium.plugins import HeatMap
import folium
from folium.plugins import MarkerCluster

In [ ]:
df_authors = pd.read_csv('/content/gdrive/MyDrive/data_thesis/KUL_authors.csv', sep="\t")
df_hist = pd.read_csv('/content/gdrive/MyDrive/data_thesis/KUL_hist.csv', sep="\t")
df_hist_2 = pd.read_csv('/content/gdrive/MyDrive/data_thesis/KUL_hist_2.csv', sep="\t")
df_user = pd.read_csv('/content/gdrive/MyDrive/data_thesis/KUL_user.csv', sep="\t")
df_user_2 = pd.read_csv('/content/gdrive/MyDrive/data_thesis/KUL_user_2.csv', sep="\t")
df_books = pd.read_csv('/content/gdrive/MyDrive/data_thesis/KUL_books_item_type_added.csv', sep="\t")
df_topics = pd.read_csv('/content/gdrive/MyDrive/data_thesis/KUL_topics.csv', sep="\t")


In [ ]:
# Historical book loan data
data_hist = pd.concat([df_hist, df_hist_2])
data_hist = data_hist.merge(df_books[['exem_id','book_title','original_title', 'isbn', 'item_type']], on='exem_id', how='inner')


## Data Cleaning

In [ ]:
# Exclude non-individual and special filter types
data_hist = data_hist[~data_hist['membership_type'].isin([62, 100, 120, 126, 151, 158, 181, 210, 211])] 

# Exclude the library user
data_hist =  data_hist[data_hist['actor_id'] !=0 ]

# Exclude this interlibrary loan book
data_hist =  data_hist[data_hist['book_title'] != 'Interbibliothecair leenverkeer Leuven' ]

## Data Preprocessing

In [ ]:
# Create time and date variables
data_hist['transaction_date_datetime'] = pd.to_datetime(data_hist['transaction_date'])
data_hist['transaction_day'] =  data_hist['transaction_date_datetime'].dt.day
data_hist['transaction_month'] =  data_hist['transaction_date_datetime'].dt.month
data_hist['transaction_year_month'] = data_hist['transaction_date_datetime'].dt.to_period('M')

In [ ]:
# Filter for book loans
data_hist_loans = data_hist[data_hist['transaction_type'] == 1]

# Temporal Analysis

## Book loans by year and month

In [ ]:
# Book loans by year and month
book_loans_year_month = data_hist_loans.groupby(['transaction_year_month'])['exem_id'].count().reset_index(name ='count_exem_id')
book_loans_year_month['month'] =  book_loans_year_month['transaction_year_month'].dt.month
 

In [ ]:
# Convert the date column to datetime objects
book_loans_year_month['date_format'] = book_loans_year_month['transaction_year_month'].apply(lambda x: x.to_timestamp())

In [ ]:
book_loans_year_month['count_exem_id'].median()

30805.0

In [ ]:
# Create a Plotly figure
fig = go.Figure()

# Add a line trace to the figure
fig.add_trace(go.Scatter(x=book_loans_year_month['date_format'], y=book_loans_year_month['count_exem_id'], mode='lines'))

# Set the x-axis title
fig.update_xaxes(title_text='Date')

# Set the y-axis title
fig.update_yaxes(title_text='Book loans')

# Add a title to the figure
#fig.update_layout(title_text='Monthly evolution of book loans')

# Show the plot
fig.show()

## Median of book loans by month

In [ ]:
# Monthly median of book loans
seasonality_book_loans = book_loans_year_month.groupby('month')['count_exem_id'].median().reset_index(name ='mean_exem_id')

In [ ]:
# Create a Plotly figure
fig_season = go.Figure()

# Add a line trace to the figure
fig_season.add_trace(go.Bar(x=seasonality_book_loans['month'], y=seasonality_book_loans['mean_exem_id'] ))

# Set the x-axis title
fig_season.update_xaxes(title_text='Month')

# Set the y-axis title
fig_season.update_yaxes(title_text='Avg book loans')

# Add a title to the figure
#fig_season.update_layout(title_text='Monthly average book loans')

# Show the plot
fig_season.show()

## Median book loans by day of the month

In [ ]:
book_loans_daily_basis = data_hist_loans.groupby(['transaction_date_datetime'])['exem_id'].count().reset_index(name = 'exem_id')
book_loans_daily_basis['day'] =  book_loans_daily_basis['transaction_date_datetime'].dt.day
book_loans_daily_basis['day_of_week'] =  book_loans_daily_basis['transaction_date_datetime'].dt.dayofweek 
book_loans_daily_basis['day_of_week_name'] = book_loans_daily_basis['transaction_date_datetime'].dt.strftime('%A')
book_loans_daily_basis_month = book_loans_daily_basis.groupby('day')['exem_id'].median().reset_index(name ='mean_exem_id')

In [ ]:
# Create a Plotly figure
fig_season = go.Figure()

# Add a line trace to the figure
fig_season.add_trace(go.Bar(x=book_loans_daily_basis_month['day'], y=book_loans_daily_basis_month['mean_exem_id'] ))

# Set the x-axis title
fig_season.update_xaxes(title_text='Month')

# Set the y-axis title
fig_season.update_yaxes(title_text='Avg Quantity')

# Add a title to the figure
fig_season.update_layout(title_text='Median book loans')

# Show the plot
fig_season.show()

## Median book loans by day of the week

In [ ]:
book_loans_daily_basis_d_o_w = book_loans_daily_basis.groupby(['day_of_week_name', 'day_of_week'])['exem_id'].median().reset_index(name ='mean_exem_id')
# Sort the dataframe by day_of_week column
book_loans_daily_basis_d_o_w = book_loans_daily_basis_d_o_w.sort_values('day_of_week')


In [ ]:
# Create a Plotly figure
fig_season = go.Figure()

# Add a line trace to the figure
fig_season.add_trace(go.Bar(x=book_loans_daily_basis_d_o_w['day_of_week_name'], y=book_loans_daily_basis_d_o_w['mean_exem_id'] ))

# Set the x-axis title
fig_season.update_xaxes(title_text='Day of the week')

# Set the y-axis title
fig_season.update_yaxes(title_text='Avg book loans')

# Add a title to the figure
#fig_season.update_layout(title_text='Average book loans by day of the week')

# Show the plot
fig_season.show()

# Loan Duration

In [ ]:
# Book loans. This dataset contains the book loans and returns. df_hist_1 does not have returns data
df_hist_book = df_hist.merge(df_books[['exem_id','book_title','original_title', 'isbn','collation']], on='exem_id', how='inner')

In [ ]:
# Filter by book loans and returns
df_hist_book_1 = df_hist_book[(df_hist_book['transaction_type'].isin([1,2]))].sort_values(['actor_id','book_title','transaction_date'])

In [ ]:
# Change format of the transaction date variable
df_hist_book_1['transaction_date_format'] = pd.to_datetime(df_hist_book_1['transaction_date'], format='%Y/%m/%d')

In [ ]:
# Sort the transactions by date within each group
transactions = df_hist_book_1.sort_values(['actor_id', 'book_title', 'transaction_date_format'])
# Group the transactions by item ID and title
groups = transactions.groupby(['actor_id',  'book_title'])

# Calculate the duration between consecutive dates in each group
durations = groups['transaction_date_format'].diff().dt.days

# Combine the durations with the original transactions DataFrame
df_hist_book_1['duration'] = durations

In [ ]:
# If it's a loan, assign an NA to the 'duration' column
df_hist_book_1.loc[df_hist_book_1['transaction_type'] == 1, 'duration'] = np.nan

In [ ]:
# There are users that have multiple membership types. The user had membership type "60", then returned the book having a membership type "62" (excluded from the analysis), then had again a membership type of "60" 
#               EXAMPLE: 
# Person changed their membership type. 1 transaction is not showned in this table because membership_type 100 is excluded
# hist_1_book_1[(hist_1_book_1['actor_id']== 1902529) & (hist_1_book_1['book_title']== 'Wat gaat het kindje doen?') ].sort_values('transaction_date')
# The user had a different membership type
# data_hist_dur[(data_hist_dur['actor_id']== 1902529) & (data_hist_dur['book_title']== 'Wat gaat het kindje doen?') ].sort_values('transaction_date')

# Exclude non-individual and special filter types
df_hist_book_1 = df_hist_book_1[~df_hist_book_1['membership_type'].isin([62, 100, 120, 126, 151, 158, 181, 210, 211])] 

# Exclude the library user
df_hist_book_1 =  df_hist_book_1[df_hist_book_1['actor_id'] !=0 ]

# Exclude this interlibrary loan book
df_hist_book_1 =  df_hist_book_1[df_hist_book_1['book_title'] != 'Interbibliothecair leenverkeer Leuven' ]

In [ ]:
# User that loaned a book but never returned it
# df_hist_book_1[(df_hist_book_1['actor_id']== 1944696) &(df_hist_book_1['book_title']== 'De freule en de Führer : het verhaal van Unity Mitford')  ]   

In [ ]:
# User that loaned a book, extended the loan and returned the book
# data_hist_dur = pd.concat([df_hist, df_hist_2]).merge(df_books[['exem_id','book_title','original_title', 'isbn']], on='exem_id', how='inner')
# data_hist_dur[(data_hist['actor_id']== 1944696) &(data_hist_dur['book_title']== 'Het Heksenhotel')  ]   

In [ ]:
# Summary of the
df_hist_book_1['duration'].describe()

count    607398.000000
mean         32.053894
std          21.073424
min           0.000000
25%          17.000000
50%          28.000000
75%          42.000000
max         229.000000
Name: duration, dtype: float64

In [ ]:
# User with the highest book loan duration
# hist_1_book_1[hist_1_book_1['duration'] == 229]

In [ ]:
# User that has loaned the same book twice but just returned it once
# hist_1_book_1[(hist_1_book_1['actor_id']== 1931199) & (hist_1_book_1['book_title']== 'Die dag aan zee') ].sort_values('transaction_date')

In [ ]:
duration_book_loans = df_hist_book_1[-df_hist_book_1['duration'].isna()] 
duration_book_loans.head(2)

,actor_id,exem_id,titelnr,transaction_type,transaction_date,due_date,orig_due_date,membership_type,too_late,book_title,original_title,isbn,collation,transaction_date_format,duration
287604,21195,16880031,1358023,2,2022-04-02,2022-04-02,2022-03-26,60,-7,Catherine,Northanger abbey,9789076542928,"271 p. : ill., z/w",2022-04-02,35.0
660752,21195,16917763,1074871,2,2022-09-06,2022-09-06,2022-08-25,60,-12,De glazen troon,Throne of glass,9789022570036,359 p. : ill.,2022-09-06,40.0


In [ ]:
# Define bin edges and labels
bins = list(range(0, 101, 5)) + [float('inf')]
labels = [f"{i}-{i+4}" for i in bins[:-2]]
labels.append('>100')

# Create bins using pd.cut()
duration_book_loans['duration_bins'] = pd.cut(duration_book_loans['duration'], bins=bins, labels=labels, include_lowest=True)

<ipython-input-32-201a354f364e>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Books returned before 28 days
350662/ (350662+256736 )

0.5773183316375754

In [ ]:
# Books returned after 28 days
1- (350662/ (350662+256736 ))

0.4226816683624246

In [ ]:
# Book loans duration by bins of days
duration_book_loans_count = duration_book_loans.groupby('duration_bins')['actor_id'].count().reset_index(name = 'count_loans')
duration_book_loans_count['percentage'] = ((duration_book_loans_count['count_loans'] / duration_book_loans['actor_id'].count())*100) 
duration_book_loans_count.head(2)

,duration_bins,count_loans,percentage
0,0-4,28559,4.701859
1,5-9,46489,7.653795


In [ ]:
duration_book_loans_count

,duration_bins,count_loans,percentage
0,0-4,28559,4.701859
1,5-9,46489,7.653795
2,10-14,60205,9.911952
3,15-19,50762,8.357288
4,20-24,74392,12.247653
5,25-29,111927,18.427291
6,30-34,42605,7.014346
7,35-39,26511,4.364683
8,40-44,26635,4.385098
9,45-49,23116,3.805742


In [ ]:
# Create a Plotly figure
fig_season = go.Figure()

# Add a line trace to the figure
fig_season.add_trace(go.Bar(x=duration_book_loans_count['duration_bins'], y=duration_book_loans_count['percentage'] ))

# Set the x-axis title
fig_season.update_xaxes(title_text='Duration')

# Set the y-axis title
fig_season.update_yaxes(title_text='Percentage of Loans')

# Add a title to the figure
#fig_season.update_layout(title_text='Avg duration of book loans')

# Show the plot
fig_season.show()

# Popular books

In [ ]:
# Define the conditions and corresponding values
adult_books = ['B2', 'B3', 'K2', 'S2','ZV', 'ZW']
youth_books = ['B4', 'B5', 'BA',   'S4', 'ZJ']

conditions = [
    data_hist_loans['item_type'].isin(adult_books),
    data_hist_loans['item_type'].isin(youth_books)
]

choices = ['Adult Book', 'Youth Book']

# Use np.where to create the if-else condition
data_hist_loans['book_type'] = np.where(conditions[0], choices[0], np.where(conditions[1], choices[1], 'Other Category'))


<ipython-input-49-a379eaf56319>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Add the author to the dataframe
book_loans_w_author = data_hist_loans.merge(df_authors[df_authors['author_type']=='primary_author'].drop_duplicates(), on ='titelnr', how ='left')
 

In [ ]:
# Books per category
data_hist_loans.groupby(['book_type'])['book_type'].count()

book_type
Adult Book        711434
Other Category      4642
Youth Book        580571
Name: book_type, dtype: int64

In [ ]:
data_hist_loans.head(1)

,actor_id,exem_id,titelnr,transaction_type,transaction_date,due_date,orig_due_date,membership_type,too_late,book_title,original_title,isbn,item_type,transaction_date_datetime,transaction_day,transaction_month,transaction_year_month,book_type
0,1860725,16828044,1139551,1,2020-12-09,1900-01-01,1900-01-01,20,0,Ik geef je de zon,I'll give you the sun,9789020679779,BA,2020-12-09,9,12,2020-12,Youth Book


In [ ]:
# Top10 youth books
top_10_youth = data_hist_loans[data_hist_loans['book_type']== 'Youth Book'].groupby(['titelnr'])['titelnr'].count().reset_index(name='qty').sort_values( by =['qty'], ascending = False).head(10)
top_10_youth.merge(data_hist_loans[['titelnr','book_title']].drop_duplicates(), on ='titelnr', how ='left'  )

,titelnr,qty,book_title
0,1355122,448,De waanzinnige boomhut van 104 verdiepingen
1,1242867,435,De waanzinnige boomhut van 78 verdiepingen
2,1293643,413,De waanzinnige boomhut van 91 verdiepingen
3,1151354,403,De waanzinnige boomhut van 52 verdiepingen
4,1107090,392,De waanzinnige boomhut van 39 verdiepingen
5,1190859,384,De waanzinnige boomhut van 65 verdiepingen
6,1313233,354,Wegwezen
7,1204771,350,FF offline
8,1326531,329,Liefdeskriebels : avonturen van een niet-zo-ge...
9,1255270,324,Drie keer niks


In [ ]:
# Top10 authors of youth books 
book_loans_w_author[book_loans_w_author['book_type']== 'Youth Book'].groupby(['author_name'])['author_name'].count().reset_index(name='qty').sort_values( by =['qty'], ascending = False).head(10)

,author_name,qty
5878,"Stilton, Geronimo",13372
6410,"Van Loon, Paul",9328
5684,"Slegers, Liesbet",7895
123,"Amant, Kathleen",7155
5879,"Stilton, Thea",6634
1336,"Dahl, Roald",5967
6348,"Van Genechten, Guido",5634
3397,"Kinney, Jeff",5386
6619,"Vanden Heede, Sylvia",5287
466,"Berebrouckx, Annemie",4296


In [ ]:
# Top10 adult books
top_10_adult = data_hist_loans[data_hist_loans['book_type']== 'Adult Book'].groupby(['titelnr'])['titelnr'].count().reset_index(name='qty').sort_values( by =['qty'], ascending = False).head(10)
top_10_adult.merge(data_hist_loans[['titelnr','book_title']].drop_duplicates(), on ='titelnr', how ='left'  )

,titelnr,qty,book_title
0,1356149,622,Een tragisch verhaal
1,1294615,607,"Want alles gaat voorbij, maar niets gaat over"
2,1270738,585,Het gewicht van de haat
3,1744831,528,De zeven zussen : Maia's verhaal
4,4403777,454,Storm : Ally's verhaal
5,1239697,454,Over elk vergeten heen
6,4403600,397,Schaduw : Stars verhaal
7,1335670,390,Grand Hotel Europa : roman
8,1191095,388,Voor wie de klok slaat
9,1305102,375,Parel : CeCe's verhaal


In [ ]:
# Top10 authors of adult books 
book_loans_w_author[book_loans_w_author['book_type']== 'Adult Book'].groupby(['author_name'])['author_name'].count().reset_index(name='qty').sort_values( by =['qty'], ascending = False).head(10)

,author_name,qty
5038,"Claes, Jo",7103
23340,"Roberts, Nora",6294
23249,"Riley, Lucinda",4987
5496,"Coppers, Toni",4336
10197,"French, Nicci",3788
916,"Aspe, Pieter",3409
25451,"Slaughter, Karin",2940
19596,"Montefiore, Santa",2708
26122,"Steel, Danielle",2476
7501,"Deflo, Luc",2254


# Map

In [ ]:
# Users dataset
data_user = pd.concat([df_user, df_user_2])
data_user['zipcode_new'] = data_user['zipcode'].str.extract('(\d+)', expand=False) 
data_user['street_township'] = data_user['street'] + ", " + data_user['township'].str.capitalize()
data_user['township_c'] = data_user['township'].str.capitalize()
data_user.head(2)

,actor_id,birth_date,sex,street,number,township,zipcode,zipcode_new,street_township,township_c
0,1860714,1973-07-08,V,Lod. van Veltemstraat,63,HERENT,3020,3020,"Lod. van Veltemstraat, Herent",Herent
1,1860725,2006-03-12,V,Bovenstraat,1,LUBBEEK,3210,3210,"Bovenstraat, Lubbeek",Lubbeek


In [ ]:
# Add new variable with the users' address
data_user_clean = data_user.drop_duplicates()

data_user_clean['number_new'] = data_user_clean['number'].str.split("/").str[0]

data_user_clean['street_number'] = data_user_clean['street'] + " " + data_user_clean['number_new']  

# Address of every user
readers_by_street_num = data_user_clean[['street_number', 'actor_id']]
readers_by_street_num.drop_duplicates(inplace = True)

<ipython-input-72-d70e7f53d3b7>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-d70e7f53d3b7>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-d70e7f53d3b7>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Qty of readers by each street
qty_readers_by_street_num = readers_by_street_num.groupby('street_number')['actor_id'].nunique().reset_index(name='q_reader')
qty_readers_by_street_num.head(2)


,street_number,q_reader
0,rue Colonel Chaltin 10,1
1,' t Pachthof 12,1


In [ ]:
# Qty of book loans per reader
actor_hist_loans =  data_hist_loans.groupby(['actor_id'])['exem_id'].count().reset_index(name ='q_exem_id')
readers_by_street_num = readers_by_street_num.merge(actor_hist_loans, on = 'actor_id', how = 'inner')
readers_by_street_num.head(2)

,street_number,actor_id,q_exem_id
0,Lod. van Veltemstraat 63,1860714,19
1,Bovenstraat 1,1860725,198


In [ ]:
# street number by loans 
readers_by_street_num.merge(actor_hist_loans, on = 'actor_id', how = 'inner')

,street_number,actor_id,q_exem_id_x,q_exem_id_y
0,Lod. van Veltemstraat 63,1860714,19,19
1,Bovenstraat 1,1860725,198,198
2,Langeveld 152,1860787,192,192
3,Dagobertstraat 57,1860855,5,5
4,Klein-Rijselstraat 30,1860922,101,101
...,...,...,...,...
31184,Hoogland 126,2054704,2,2
31185,Arnoudt Rulenslaan 22,1977169,1,1
31186,Leuvensesteenweg 23,2055690,2,2
31187,Langendaallaan 104,1904909,1,1


In [ ]:
# Qty of book loans per street
qty_books_loaned_by_street_num = readers_by_street_num.groupby('street_number')['q_exem_id'].sum().reset_index(name='q_exem_id')
qty_books_loaned_by_street_num.head(2)

,street_number,q_exem_id
0,rue Colonel Chaltin 10,3
1,' t Pachthof 12,9


In [ ]:
# Qty of book loans and readers per street
summary_street_data_to_map = qty_readers_by_street_num.merge(qty_books_loaned_by_street_num, on = 'street_number', how = 'left')
summary_street_data_to_map.head(2)

,street_number,q_reader,q_exem_id
0,rue Colonel Chaltin 10,1,3.0
1,' t Pachthof 12,1,9.0


In [ ]:
# Shapefile
gdf = gpd.read_file('/content/gdrive/MyDrive/data_thesis/municipalities')

# Print the first few rows to make sure the data was loaded correctly
gdf.head()

,OBJECTID,ADMUNAFR,ADMUNADU,ADMUNAGE,Communes,CODE_INS,arrond,geometry
0,1,AARTSELAAR,AARTSELAAR,AARTSELAAR,Aartselaar,11001,11,"POLYGON ((4.40125 51.14814, 4.40114 51.14797, ..."
1,2,ANVERS,ANTWERPEN,ANTWERPEN,Antwerpen,11002,11,"POLYGON ((4.34109 51.35766, 4.34112 51.35760, ..."
2,3,BOECHOUT,BOECHOUT,BOECHOUT,Boechout,11004,11,"POLYGON ((4.52882 51.19051, 4.52971 51.19020, ..."
3,4,BOOM,BOOM,BOOM,Boom,11005,11,"POLYGON ((4.36411 51.10597, 4.36456 51.10596, ..."
4,5,BORSBEEK,BORSBEEK,BORSBEEK,Borsbeek,11007,11,"POLYGON ((4.48350 51.20315, 4.48354 51.20314, ..."


In [ ]:
# The shapemap has the coordinates inverted, this code aims to correct that
for idx,row in gdf.iterrows():
  if type(gdf.geometry.iloc[idx]) == Polygon:
    exterior_coords = gdf.geometry.iloc[idx].exterior.coords
    reversed_coords = [(coord[1], coord[0]) for coord in exterior_coords[::-1]]
    new_exterior = Polygon(reversed_coords)
    gdf.geometry.iloc[idx] = new_exterior


In [ ]:
# import address scraped
address_lat_lon_0 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_1.csv")
address_lat_lon_1 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_2.csv")
address_lat_lon_2 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_3.csv")
address_lat_lon_3 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_4.csv")
address_lat_lon_4 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_5.csv")
address_lat_lon_5 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_6.csv")
address_lat_lon_6 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_7.csv")
address_lat_lon_7 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_leuven_lat_lon_8.csv")

address_lat_lon_8 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_no_leuven_lat_lon_1.csv")
address_lat_lon_9 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_no_leuven_lat_lon_2.csv")
address_lat_lon_10 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_no_leuven_lat_lon_3.csv")
address_lat_lon_11 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_no_leuven_lat_lon_4.csv")
address_lat_lon_12 = pd.read_csv("/content/gdrive/MyDrive/data_thesis/map/address_no_leuven_lat_lon_5.csv")

In [ ]:
# concatenate address scraped
address_lat_lon_gen = pd.concat([address_lat_lon_0, address_lat_lon_1, address_lat_lon_2, address_lat_lon_3,
           address_lat_lon_4, address_lat_lon_5, address_lat_lon_6, address_lat_lon_7,
           address_lat_lon_8, address_lat_lon_9, address_lat_lon_10, address_lat_lon_11, address_lat_lon_12],axis = 0) 

address_lat_lon_gen = address_lat_lon_gen[['Street',	'Latitude',	'Longitude']]  
address_lat_lon_gen.dropna(inplace=True)
address_lat_lon_gen.drop_duplicates(inplace = True)
address_lat_lon_gen.head(2)

,Street,Latitude,Longitude
0,Dagobertstraat 57,50.87564,4.71447
1,Raadhuislaan 9,50.88841,4.73501


In [ ]:
# Readers and book loans by every longitude/latitude
user_lat_lon_g = readers_by_street_num.merge(address_lat_lon_gen, left_on ='street_number', right_on = 'Street', how='left')
user_lat_lon_g.head(2) 

,street_number,actor_id,q_exem_id,Street,Latitude,Longitude
0,Lod. van Veltemstraat 63,1860714,19,Lod. van Veltemstraat 63,50.89906,4.60905
1,Bovenstraat 1,1860725,198,Bovenstraat 1,51.05546,3.57631


In [ ]:
# Exclude streets without a latitude/longitude
data_to_map = user_lat_lon_g[-user_lat_lon_g['street_number'].isnull()]

In [ ]:
# Tranform the latitude/longitude to geometry format
data_to_map['geometry'] = [Point(xy) for xy in zip(data_to_map['Latitude'], data_to_map['Longitude'])]
data_to_map.head(2)

<ipython-input-84-bb74b07ac528>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,street_number,actor_id,q_exem_id,Street,Latitude,Longitude,geometry
0,Lod. van Veltemstraat 63,1860714,19,Lod. van Veltemstraat 63,50.89906,4.60905,POINT (50.89906 4.60905)
1,Bovenstraat 1,1860725,198,Bovenstraat 1,51.05546,3.57631,POINT (51.05546 3.57631)


In [ ]:
# create a GeoDataFrame with the lat/long coordinates
geo_df = gpd.GeoDataFrame(data_to_map, geometry=data_to_map['geometry'], crs=gdf.crs)
geo_df.head(2)

,street_number,actor_id,q_exem_id,Street,Latitude,Longitude,geometry
0,Lod. van Veltemstraat 63,1860714,19,Lod. van Veltemstraat 63,50.89906,4.60905,POINT (50.89906 4.60905)
1,Bovenstraat 1,1860725,198,Bovenstraat 1,51.05546,3.57631,POINT (51.05546 3.57631)


In [ ]:
# spatial join the two GeoDataFrames
joined_gdf = gpd.sjoin(geo_df, gdf, how='inner', predicate='within')
joined_gdf.head(2)

,street_number,actor_id,q_exem_id,Street,Latitude,Longitude,geometry,index_right,OBJECTID,ADMUNAFR,ADMUNADU,ADMUNAGE,Communes,CODE_INS,arrond
0,Lod. van Veltemstraat 63,1860714,19,Lod. van Veltemstraat 63,50.89906,4.60905,POINT (50.89906 4.60905),134,135,HERENT,HERENT,HERENT,Herent,24038,24
18,René Eversstraat 25,1893675,2,René Eversstraat 25,50.90896,4.66327,POINT (50.90896 4.66327),134,135,HERENT,HERENT,HERENT,Herent,24038,24


In [ ]:
# Qty of readers and book loans by latitude/longitude
summary_heatmap = joined_gdf.groupby(['Latitude', 'Longitude']).agg({'actor_id': 'count', 'q_exem_id': 'sum'}).reset_index()
summary_heatmap.head(2)

,Latitude,Longitude,actor_id,q_exem_id
0,49.55218,5.46575,2,7
1,49.56675,5.82865,1,8


In [ ]:
# Format variables
summary_heatmap['q_reader_float'] = summary_heatmap['actor_id'].astype(float)
summary_heatmap['q_exem_id_float'] = summary_heatmap['q_exem_id'].astype(float)

In [ ]:
m = folium.Map(location = [50.5039, 4.4699], tiles = 'OpenStreetMap',                                 
          zoom_start=6)

# Create separate MarkerCluster objects for readers and loans
reader_cluster = MarkerCluster(name='Readers').add_to(m)
 

# Create a list of tuples containing the location and intensity data
heat_data_readers = []
heat_data_q_exem_id = []
for i, row in summary_heatmap.iterrows():
    lat = summary_heatmap.at[i, 'Latitude']
    lng = summary_heatmap.at[i, 'Longitude']
    intensity_qty_readers = summary_heatmap.at[i, 'q_reader_float']
    intensity_qty_loans = summary_heatmap.at[i, 'q_exem_id_float']
    heat_data_readers.append((lat, lng, intensity_qty_readers))
    heat_data_q_exem_id.append((lat, lng, intensity_qty_loans))

# Add the heat map layer to the map
HeatMap(heat_data_readers, name='Readers - Heatmap', gradient={0.4: 'blue', 0.6: 'lime', 1: 'red'},
        blur=10, min_opacity=0.2, show=False ).add_to(m)
 
# Add the heat map layer to the map
HeatMap(heat_data_q_exem_id, name='Qty loans - Heatmap', gradient={0.4: 'blue', 0.6: 'lime', 1: 'red'},
        blur=10, min_opacity=0.2, show=False ).add_to(m)
 
# Icon for the library
folium.Marker([50.8808, 4.7037], 
              popup='Leuven Public Library',
              icon=folium.Icon(color='red',icon='university', prefix='Lib') 
             ).add_to(m)

for i, row in joined_gdf.iterrows():
    lat = joined_gdf.at[i, 'Latitude']
    lng = joined_gdf.at[i, 'Longitude']
    q_reader = joined_gdf.at[i, 'actor_id']
    q_exem_id = joined_gdf.at[i, 'q_exem_id']
    popup = 'Reader: ' + str(q_reader) + '<br>' + 'Qty Loans: ' + str(q_exem_id) 
    #popup_1 =  'Qty Loans: ' + str(q_exem_id) 
    folium.Marker(location=[lat, lng], popup=popup, icon=folium.Icon(color='blue', icon='user')).add_to(reader_cluster)
    #folium.Marker(location=[lat, lng], popup=popup_1, icon=folium.Icon(color='red', icon='book')).add_to(loan_cluster)

 
 
folium.LayerControl().add_to(m)

m.save('/content/gdrive/MyDrive/data_thesis/map/readers_library_new.html')

In [ ]:
summary_heatmap['actor_id'].sum()

30948

In [ ]:
# Readers in the surroundings of Leuven
12036/30948

0.3889104303993796